In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os, os.path
import statistics
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings('ignore')

In [3]:
jobzone_data = pd.read_csv("Compiled_employment_change_jobzone.csv")
jobzone_data = jobzone_data.set_index("state")
jobzone_data

,1,2,3,4,5,emissions,scenario,analysis
state,,,,,,,,
AL,1.347622,0.509689,0.774484,0.629470,0.524289,NNP,Low_RE_Cost,v6.2jedi_mid
AR,2.431044,1.245582,1.391552,0.918893,0.736931,NNP,Low_RE_Cost,v6.2jedi_mid
AZ,1.873082,1.237740,1.199898,0.889443,0.700864,NNP,Low_RE_Cost,v6.2jedi_mid
CA,1.684446,1.415953,1.397666,1.081018,0.732937,NNP,Low_RE_Cost,v6.2jedi_mid
CO,0.180605,0.643272,0.591262,0.739429,0.441324,NNP,Low_RE_Cost,v6.2jedi_mid
...,...,...,...,...,...,...,...,...
VT,5.937824,3.723443,3.246021,2.373214,1.533399,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2jedi_low
WA,4.651589,1.380936,1.229090,0.951697,0.795918,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2jedi_low
WI,3.071157,1.540507,1.777235,1.056433,0.741334,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2jedi_low


In [4]:
ems_list = ["NNP","95% by 2050","95% by 2035"]
add_name = [""," 95 by 2050"," 95 by 2035"]
for zone in np.arange(5)+1:
    jobzone_data_now = jobzone_data.rename(columns={str(zone):"MismatchMult"})
    for this_ems in ems_list:
        mid_name = "Mid Case"+add_name[ems_list.index(this_ems)]
        mid_name = mid_name.replace(" ","_")
        count = 0
        for this_state in state_coords["state"].to_list():
            select_data = jobzone_data_now.query("emissions==@this_ems").loc[this_state]
            select_data = select_data.reset_index()
            max_ind = select_data['MismatchMult'].idxmax()
            min_ind = select_data['MismatchMult'].idxmin()
            mid_val = select_data.query("scenario==@mid_name").query("analysis=='v6.2jedi_mid'")["MismatchMult"].iloc[0]
            if mid_val > 0:
                not_unanimous = int(any(select_data["MismatchMult"] < 0))
                majority = sum([int(s>0) for s in select_data["MismatchMult"]]) / len(select_data["MismatchMult"])
            else:
                not_unanimous = int(any(select_data["MismatchMult"] > 0))
                majority = sum([int(s<0) for s in select_data["MismatchMult"]]) / len(select_data["MismatchMult"])
            more_select_data = select_data.iloc[[max_ind,min_ind],:]
            more_select_data["Type"] = ["max","min"]
            more_select_data["std"] = select_data["MismatchMult"].std()
            more_select_data["mean"] = select_data["MismatchMult"].mean()
            more_select_data["diff"] = not_unanimous
            more_select_data["majority"] = majority
            more_select_data["mid"] = mid_val
            if count == 0:
                extreme_data = more_select_data
            else:
                extreme_data = extreme_data.append(more_select_data)
            count = count + 1
        extreme_data_min = extreme_data.query("Type=='min'").set_index("state")
        extreme_data_max = extreme_data.query("Type=='max'").set_index("state")

        states_extreme = state_coords.copy().set_index("state")
        states_extreme["max_scenario"] = extreme_data_max["scenario"]
        states_extreme["max_analysis"] = extreme_data_max["analysis"]
        states_extreme["max_value"] = extreme_data_max["MismatchMult"]
        states_extreme["min_scenario"] = extreme_data_min["scenario"]
        states_extreme["min_analysis"] = extreme_data_min["analysis"]
        states_extreme["min_value"] = extreme_data_min["MismatchMult"]
        states_extreme["std"] = extreme_data_min["std"]
        states_extreme["mean"] = extreme_data_min["mean"]
        states_extreme = states_extreme.drop(columns=["row","col"])

        states_extreme["mid_value"] = extreme_data_min["mid"]
        states_extreme["diff"] = extreme_data_min["diff"]
        states_extreme["majority"] = extreme_data_min["majority"]
        states_extreme.to_csv("net_jobzone_employment"+add_name[ems_list.index(this_ems)].replace(" ","_")+"_"+str(zone)+".csv")
states_extreme

,max_scenario,max_analysis,max_value,min_scenario,min_analysis,min_value,std,mean,mid_value,diff,majority
state,,,,,,,,,,,
WA,Electrification_EnhancedFlex_95_by_2035,v6.2jedi_low,1.021984,High_RE_Cost_95_by_2035,v6.2jedi_high,0.698030,0.085743,0.820397,0.839425,0,1.0000
OR,Low_RE_Cost_95_by_2035,v6.2jedi_mid,1.072200,Mid_Case_Base_Flex_95_by_2035,v6.2jedi_low,0.757169,0.099300,0.883224,0.764188,0,1.0000
CA,Electrification_EnhancedFlex_95_by_2035,v6.2jedi_high,3.204904,Low_Trans_95_by_2035,v6.2jedi_low,0.898268,0.578466,1.717178,1.638484,0,1.0000
ID,Electrification_95_by_2035,v6.2jedi_high,2.068218,Low_Demand_Growth_95_by_2035,v6.2jedi_low,0.738204,0.363824,0.954508,0.793905,0,1.0000
NV,High_RE_Cost_95_by_2035,v6.2jedi_low,1.762269,Low_RE_Cost_95_by_2035,v6.2jedi_high,0.726216,0.266298,1.153056,1.008001,0,1.0000
UT,Electrification_95_by_2035,v6.2jedi_low,0.632682,Low_RE_Cost_95_by_2035,v6.2jedi_high,0.169093,0.093735,0.322559,0.289977,0,1.0000
AZ,Low_Nuclear_CCS_Cost_95_by_2035,v6.2jedi_high,1.474326,High_RE_Cost_95_by_2035,v6.2jedi_high,0.708521,0.187396,0.831369,0.801656,0,1.0000
MT,Electrification_95_by_2035,v6.2jedi_low,1.956133,High_Trans_95_by_2035,v6.2jedi_high,0.320056,0.367296,0.794225,0.970674,0,1.0000
WY,Electrification_EnhancedFlex_95_by_2035,v6.2jedi_low,0.067590,Low_Everything_95_by_2035,v6.2jedi_high,-0.186518,0.065949,-0.108336,-0.138329,1,0.8750
